<a href="https://colab.research.google.com/github/bhanuteja2001/COVID-19-XrayDetection/blob/main/Datathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/My Drive/covid_image_data.zip" -d "/content/drive/My Drive/"

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_training = pd.read_csv('/content/drive/MyDrive/covid_image_data/Training_set_covid.csv')
df_training.head()

,filename,label
0,Image_1.jpg,1
1,Image_2.jpg,0
2,Image_3.jpg,0
3,Image_4.jpg,0
4,Image_5.jpg,0


In [ ]:
df_testing = pd.read_csv('/content/drive/MyDrive/covid_image_data/Testing_set_covid.csv')
df_testing.head()

,filename
0,Image_1.jpg
1,Image_2.jpg
2,Image_3.jpg
3,Image_4.jpg
4,Image_5.jpg


In [ ]:
df_training.tail()

,filename,label
3474,Image_3475.jpg,0
3475,Image_3476.jpg,0
3476,Image_3477.jpg,0
3477,Image_3478.jpg,1
3478,Image_3479.jpg,0


In [ ]:
training_file_path = ['/content/drive/MyDrive/covid_image_data/train/' + fname[:-4] +  '.jpg' for fname in df_training['filename']]
training_file_path

['/content/drive/MyDrive/covid_image_data/train/Image_1.jpg',
 '/content/drive/MyDrive/covid_image_data/train/Image_2.jpg',
 '/content/drive/MyDrive/covid_image_data/train/Image_3.jpg',
 '/content/drive/MyDrive/covid_image_data/train/Image_4.jpg',
 '/content/drive/MyDrive/covid_image_data/train/Image_5.jpg',
 '/content/drive/MyDrive/covid_image_data/train/Image_6.jpg',
 '/content/drive/MyDrive/covid_image_data/train/Image_7.jpg',
 '/content/drive/MyDrive/covid_image_data/train/Image_8.jpg',
 '/content/drive/MyDrive/covid_image_data/train/Image_9.jpg',
 '/content/drive/MyDrive/covid_image_data/train/Image_10.jpg',
 '/content/drive/MyDrive/covid_image_data/train/Image_11.jpg',
 '/content/drive/MyDrive/covid_image_data/train/Image_12.jpg',
 '/content/drive/MyDrive/covid_image_data/train/Image_13.jpg',
 '/content/drive/MyDrive/covid_image_data/train/Image_14.jpg',
 '/content/drive/MyDrive/covid_image_data/train/Image_15.jpg',
 '/content/drive/MyDrive/covid_image_data/train/Image_16.jpg',
 

In [ ]:
testing_file_path = ['/content/drive/MyDrive/covid_image_data/test/' + fname[:-4] +  '.jpg' for fname in df_testing['filename']]
testing_file_path

['/content/drive/MyDrive/covid_image_data/test/Image_1.jpg',
 '/content/drive/MyDrive/covid_image_data/test/Image_2.jpg',
 '/content/drive/MyDrive/covid_image_data/test/Image_3.jpg',
 '/content/drive/MyDrive/covid_image_data/test/Image_4.jpg',
 '/content/drive/MyDrive/covid_image_data/test/Image_5.jpg',
 '/content/drive/MyDrive/covid_image_data/test/Image_6.jpg',
 '/content/drive/MyDrive/covid_image_data/test/Image_7.jpg',
 '/content/drive/MyDrive/covid_image_data/test/Image_8.jpg',
 '/content/drive/MyDrive/covid_image_data/test/Image_9.jpg',
 '/content/drive/MyDrive/covid_image_data/test/Image_10.jpg',
 '/content/drive/MyDrive/covid_image_data/test/Image_11.jpg',
 '/content/drive/MyDrive/covid_image_data/test/Image_12.jpg',
 '/content/drive/MyDrive/covid_image_data/test/Image_13.jpg',
 '/content/drive/MyDrive/covid_image_data/test/Image_14.jpg',
 '/content/drive/MyDrive/covid_image_data/test/Image_15.jpg',
 '/content/drive/MyDrive/covid_image_data/test/Image_16.jpg',
 '/content/drive/

In [ ]:
df_training.label.value_counts()

0    3073
1     406
Name: label, dtype: int64

In [ ]:
# Confirm if number of images is same as number of labels given
if len(df_training) == len(training_file_path):
    print('Number of labels i.e. ', len(df_training), 'matches the number of filenames i.e. ', len(training_file_path))
else:
    print('Number of labels doesnot matches the number of filenames')

Number of labels i.e.  3479 matches the number of filenames i.e.  3479


In [ ]:
images_training = [[fname, '/content/drive/MyDrive/covid_image_data/train/' + fname[:-4] + '.jpg'] for fname in df_training['filename']]
images_training = pd.DataFrame(images_training)
images_training.columns = ['filename', 'file']
images_training

,filename,file
0,Image_1.jpg,/content/drive/MyDrive/covid_image_data/train/...
1,Image_2.jpg,/content/drive/MyDrive/covid_image_data/train/...
2,Image_3.jpg,/content/drive/MyDrive/covid_image_data/train/...
3,Image_4.jpg,/content/drive/MyDrive/covid_image_data/train/...
4,Image_5.jpg,/content/drive/MyDrive/covid_image_data/train/...
...,...,...
3474,Image_3475.jpg,/content/drive/MyDrive/covid_image_data/train/...
3475,Image_3476.jpg,/content/drive/MyDrive/covid_image_data/train/...
3476,Image_3477.jpg,/content/drive/MyDrive/covid_image_data/train/...
3477,Image_3478.jpg,/content/drive/MyDrive/covid_image_data/train/...


In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping
%load_ext tensorboard

In [ ]:
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
Train = pd.merge(df_training, images_training, on='filename')
Train.drop(columns = ['filename'], inplace = True)
Train.head()

,label,file
0,1,/content/drive/MyDrive/covid_image_data/train/...
1,0,/content/drive/MyDrive/covid_image_data/train/...
2,0,/content/drive/MyDrive/covid_image_data/train/...
3,0,/content/drive/MyDrive/covid_image_data/train/...
4,0,/content/drive/MyDrive/covid_image_data/train/...


In [ ]:
y = np.array(Train['label'])

In [ ]:
train_image = []
for i in tqdm(range(Train.shape[0])):
    img = image.load_img(Train['file'][i], target_size=(28,28,1), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

100%|██████████| 3479/3479 [00:35<00:00, 97.24it/s] 


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [ ]:
y_train = y_train.reshape((-1,1))
y_test = y_test.reshape((-1,1))

In [ ]:
from tensorflow.keras.applications import MobileNetV2

In [ ]:
model2 = Sequential()
model2.add(MobileNetV2(include_top = False, weights="imagenet", input_shape=(200, 200, 3)))
model2.add(tf.keras.layers.GlobalAveragePooling2D())
model2.add(Dense(1, activation = 'sigmoid'))
model2.layers[0].trainable = False

In [ ]:
from tensorflow.keras.optimizers import RMSprop
# model.compile(loss='binary_crossentropy',
#               optimizer=RMSprop(lr=0.001),
#               metrics=['accuracy', tf.keras.metrics.AUC()])

from tensorflow.keras import optimizers


model2.compile(optimizer=RMSprop(lr=0.01), loss = 'binary_crossentropy', metrics = 'accuracy')

In [ ]:
train_image_2 = []
for i in tqdm(range(Train.shape[0])):
    img = image.load_img(Train['file'][i], target_size=(200,200,3), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    train_image_2.append(img)
X = np.array(train_image_2)

100%|██████████| 3479/3479 [00:40<00:00, 85.05it/s] 


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)
y_train = y_train.reshape((-1,1))
y_test = y_test.reshape((-1,1))

In [ ]:
EPOCHS = 100

es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, 
                                               restore_best_weights=True,
                                               verbose=1)


history = model2.fit(X_train, y_train, epochs=EPOCHS,callbacks=[es_callback], validation_data=(X_test, y_test))

Epoch 1/100
87/87 [==============================] - 101s 1s/step - loss: 0.3594 - accuracy: 0.8940 - val_loss: 0.2332 - val_accuracy: 0.9181
Epoch 2/100
87/87 [==============================] - 100s 1s/step - loss: 0.2551 - accuracy: 0.9238 - val_loss: 0.2460 - val_accuracy: 0.9080
Epoch 3/100
87/87 [==============================] - 99s 1s/step - loss: 0.1759 - accuracy: 0.9375 - val_loss: 0.2619 - val_accuracy: 0.9095
Epoch 00003: early stopping


## Image Testing

In [ ]:
images_testing = [[fname, '/content/drive/MyDrive/covid_image_data/test/' + fname[:-4] + '.jpg'] for fname in df_testing['filename']]
images_testing = pd.DataFrame(images_testing)
images_testing.columns = ['filename', 'file']
images_testing

,filename,file
0,Image_1.jpg,/content/drive/MyDrive/covid_image_data/test/I...
1,Image_2.jpg,/content/drive/MyDrive/covid_image_data/test/I...
2,Image_3.jpg,/content/drive/MyDrive/covid_image_data/test/I...
3,Image_4.jpg,/content/drive/MyDrive/covid_image_data/test/I...
4,Image_5.jpg,/content/drive/MyDrive/covid_image_data/test/I...
...,...,...
865,Image_866.jpg,/content/drive/MyDrive/covid_image_data/test/I...
866,Image_867.jpg,/content/drive/MyDrive/covid_image_data/test/I...
867,Image_868.jpg,/content/drive/MyDrive/covid_image_data/test/I...
868,Image_869.jpg,/content/drive/MyDrive/covid_image_data/test/I...


In [ ]:
test_image = []
for i in tqdm(range(images_testing.shape[0])):
    img = image.load_img(images_testing['file'][i], target_size=(200, 200, 3), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
X = np.array(test_image)

100%|██████████| 870/870 [00:10<00:00, 85.97it/s]


In [ ]:
prediction = model2.predict_classes(X)

In [ ]:

# To create Dataframe of predicted value with particular respective index
res = pd.DataFrame(prediction) #preditcions are nothing but the final predictions of your model on input features of your new unseen test data
#res.index = test_new.index # its important for comparison. Here "test_new" is your new test dataset
res.columns = ["prediction"]

# To download the csv file locally
from google.colab import files
res.to_csv('prediction_results.csv')         
files.download('prediction_results.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Using Movile Net V2 72.95081967213115


3479